In [21]:
cd /home/ubuntu/Router

/home/ubuntu/Router


/home/ubuntu/Router/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [30]:
%load_ext autoreload
%autoreload 2

from datasets import load_from_disk

from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    SentenceTransformerModelCardData,
)
from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import BinaryClassificationEvaluator

from router.embedding_router import generate_embeddings_sentence_transformer, return_top_k
import json

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("/home/ubuntu/Router/models/nomic-embed-text-v1.5-router/checkpoint-21000", trust_remote_code=True)

print(model)

<All keys matched successfully>


SentenceTransformer(
  (0): Transformer({'max_seq_length': 8192, 'do_lower_case': False, 'architecture': 'NomicBertModel'})
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
)


In [25]:

dataset_dict = load_from_disk("data/positive_pairs_train_val_test_1")
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['text1', 'text2'],
        num_rows: 773081
    })
    val: Dataset({
        features: ['text1', 'text2', 'label'],
        num_rows: 85898
    })
    test: Dataset({
        features: ['text1', 'text2', 'label'],
        num_rows: 95443
    })
})


In [35]:
tools_description_json = json.load(open("/home/ubuntu/Router/data/tool_descriptions.json"))
tools_descriptions = list(tools_description_json.values())
integrations_name = list(tools_description_json.keys())

print(f'Length of tools_descriptions: {len(tools_descriptions)}')
print(f'Length of integrations_name: {len(integrations_name)}')

Length of tools_descriptions: 16
Length of integrations_name: 16


In [40]:
tool_embeddings = generate_embeddings_sentence_transformer(model, tools_descriptions)

embeddings: 38.77ms


In [ ]:
from datasets import load_from_disk

from sentence_transformers import SentenceTransformer
from router.evals import Evals

model = SentenceTransformer("/home/ubuntu/Router/models/nomic-embed-text-v1.5-router/checkpoint-21000", trust_remote_code=True)

evals_test_500_v0 = json.load(open("/home/ubuntu/Router/data/evals_test_500_v0.json"))

for test_row in evals_test_500_v0:
    query_text = test_row['text1']
    query_text = 'Turn on dark mode'
    query_embeddings = generate_embeddings_sentence_transformer(model, [query_text])
    if len(query_embeddings.shape) == 1:
        query_embeddings = query_embeddings.reshape(1, -1)
    
    top_k_indices, top_k_scores = return_top_k(tool_embeddings, query_embeddings, 16)
    for k in range(len(tool_embeddings) - 1, -1, -1):
        tool_idx = top_k_indices[0][k]
        tool_description = tools_descriptions[tool_idx]
        integration_name = integrations_name[tool_idx]
        print(f"#{k}: Integration: {integration_name}: {tool_description} Score: {top_k_scores[0][k]}")

    break


<All keys matched successfully>


embeddings: 29.38ms
#15: Integration: Unachievable Tool: The "unachievable_task" tool indicates that the desired task cannot be accomplished using the current set of tools. It serves as a placeholder to communicate the unavailability of any functional operations for the requested task. Score: 0.9565341472625732
#14: Integration: DirectMessage Tool: DirectMessage is a tool designed for direct communication, enabling users to reply to messages directly. Its primary functionality is to facilitate immediate and straightforward responses to user messages, ensuring efficient and clear communication. When this tool is called, no other tool can be called. Score: 0.4170185923576355
#13: Integration: Files Tool: The "Files" app provides comprehensive file management capabilities, allowing users to create new files, update the contents of existing files, and move unwanted files to the trash. With these tools, users can efficiently generate, modify, and organize their digital documents within the 

In [17]:
type(tools_descriptions_embeddings)

numpy.ndarray

In [ ]:
query.shape

768